# Deploying And Deleting Endpoint 

#### You will need to replace MODEL_PACKAGE_ARN with the correct model package for your endpoint. There is a helper script to allow you to check for different MODEL_PACKAGE_ARNs 

####  you should also see model package ARNS and endpoints in SageMaker Studio in the left hand navigation column

In [100]:
# checking Model Package Arns 
import boto3


sm = boto3.client("sagemaker", region_name="us-east-1")

# Replace with your model package group name
model_package_group_name = "OliveForecastModelPackageGroup"

response = sm.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    SortBy="CreationTime",
    SortOrder="Descending"
)

for pkg in response["ModelPackageSummaryList"]:
    print(pkg["ModelPackageArn"], pkg["ModelPackageStatus"], pkg["ModelApprovalStatus"])

arn:aws:sagemaker:us-east-1:006363897224:model-package/OliveForecastModelPackageGroup/1 Completed Approved


## This section actually creates endpoints DON'T FORGET TO DELETE
#### they cost about .12c per hour so leaving them running for days adds up
#### update name and model package as needed 

In [6]:
import boto3

sm = boto3.client("sagemaker")

mp = sm.describe_model_package(ModelPackageName=MODEL_PACKAGE_ARN)

model_artifact = mp["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
image_uri = mp["InferenceSpecification"]["Containers"][0]["Image"]

print("Model artifact:", model_artifact)
print("Image URI:", image_uri)


Model artifact: s3://olive-datalake-fall2025/mlops/training/pipelines-3j2htybfobfx-TrainModels-PXGIayyuK0/output/model.tar.gz
Image URI: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3


In [105]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel

sess = sagemaker.Session()

# 🔁 Re-use the exact values you used originally:
role = sagemaker.get_execution_role()      # or your existing `role` variable
model_data = "<PUT_YOUR_MODEL_DATA_S3_URI_HERE>"  # e.g. estimator.model_data
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"

model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point="inference.py",   # the file we just edited in this notebook dir
    source_dir=".",               # current working directory
    image_uri=image_uri,
)

endpoint_name = "olive-test-v18"  # new endpoint name; change if you want

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",  # same type you used before
    endpoint_name=endpoint_name,
)

print("Deployed endpoint:", endpoint_name)


  92:         df["cost_pressure"] = 0.0
  93:     return df
  94: 
  95: # -------------------------
  96: # 4. Forecast helper
  97: 
  98: 
  99: FEATURE_COLS = [
 100:     "lag1week",
 101:     "lag2week",
 102:     "rolling3",
 103:     "rolling10",
 104:     "month",
 105:     "dayofweek",
 106:     "quarter",
 107:     "sin_week",
 108:     "cost_pressure",
 109: ]
 110: 
 111: 
 112: # -------------------------
 113: 
 114: 


In [1]:
#THIS ONE##########

import boto3
import sagemaker
from sagemaker.model import Model

# ---------------------------
# 1. Pull model artifact + image from Model Registry
# ---------------------------

MODEL_PACKAGE_ARN = (
    "arn:aws:sagemaker:us-east-1:006363897224:model-package/OliveForecastModelPackageGroup/1"
)

sm = boto3.client("sagemaker", region_name="us-east-1")

mp = sm.describe_model_package(ModelPackageName=MODEL_PACKAGE_ARN)

model_artifact = mp["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
image_uri = mp["InferenceSpecification"]["Containers"][0]["Image"]

print("Model artifact:", model_artifact)
print("Image URI:", image_uri)

# ---------------------------
# 2. Create & deploy the model with your updated inference.py
# ---------------------------

session = sagemaker.Session()
role = sagemaker.get_execution_role()

# New endpoint name so you don't clash with the old one
ENDPOINT_NAME = "olive-test-v21"   # change if you want a different name

model = Model(
    image_uri=image_uri,
    model_data=model_artifact,
    role=role,
    entry_point="inference.py",    # uses the fixed inference.py in this notebook dir
    sagemaker_session=session,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",   # same as your existing notebook
    endpoint_name=ENDPOINT_NAME,
)

print("✅ Deployed endpoint:", ENDPOINT_NAME)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Model artifact: s3://olive-datalake-fall2025/mlops/training/pipelines-3j2htybfobfx-TrainModels-PXGIayyuK0/output/model.tar.gz
Image URI: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
------!✅ Deployed endpoint: olive-test-v21


In [107]:
###########TEEESSSSSSSSSTTTTTTTTTTTTTT HEEEEEEEEEEEERRRRRRRRRRREEEEEEEEEE################
import json
import boto3

runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

# use the same payload dict you had working before
response = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(payload),   # your existing payload variable
)

print(response["Body"].read().decode("utf-8"))


[{"week_start": "2025-10-05", "forecast": 4.221478971596872, "model_used": "global_ridge"}, {"week_start": "2025-10-12", "forecast": 4.134457289590174, "model_used": "global_ridge"}, {"week_start": "2025-10-19", "forecast": 4.1348182762035295, "model_used": "global_ridge"}, {"week_start": "2025-10-26", "forecast": 4.130157346978823, "model_used": "global_ridge"}, {"week_start": "2025-11-02", "forecast": 4.123299103476007, "model_used": "global_ridge"}, {"week_start": "2025-11-09", "forecast": 4.116981937433904, "model_used": "global_ridge"}, {"week_start": "2025-11-16", "forecast": 4.108495852707409, "model_used": "global_ridge"}, {"week_start": "2025-11-23", "forecast": 4.100264162268738, "model_used": "global_ridge"}, {"week_start": "2025-11-30", "forecast": 4.091117000262589, "model_used": "global_ridge"}, {"week_start": "2025-12-07", "forecast": 4.074304531047146, "model_used": "global_ridge"}, {"week_start": "2025-12-14", "forecast": 4.05916346658504, "model_used": "global_ridge"}

In [99]:
from pathlib import Path

text = Path("inference.py").read_text(encoding="utf-8")

# Show the lines around FEATURE_COLS so we can sanity-check them
for i, line in enumerate(text.splitlines(), start=1):
    if "FEATURE_COLS" in line:
        start = max(0, i-5)
        end = i+20
        snippet = "\n".join(f"{j:4d}: {l}" for j, l in enumerate(text.splitlines()[start:end], start=start+1))
        print(snippet)
        break
else:
    print("No FEATURE_COLS found in inference.py")


 187:         temp = _create_features(temp)
 188:         temp = temp[~temp.index.duplicated(keep='last')]
 189:         temp = temp.sort_index()
 190: 
 191:         X_next = temp.loc[[next_week], FEATURE_COLS]
 192:         y_hat = float(model.predict(X_next)[0])
 193: 
 194:         preds.append({'week_start': next_week.strftime('%Y-%m-%d'), 'forecast': y_hat})
 195: 
 196:         temp.loc[next_week, 'price_usd_per_l'] = y_hat
 197:         current_index = next_week
 198: 
 199:     return preds


In [83]:
import importlib.util

spec = importlib.util.spec_from_file_location("inference", "../scripts/inference.py")
m = importlib.util.module_from_spec(spec)
spec.loader.exec_module(m)
print("Loaded!")


Loaded!


In [3]:
##CAN DELETE LATER#

import json
import pandas as pd
import boto3

runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

def call_sagemaker_endpoint(payload: dict) -> pd.DataFrame:
    response = runtime.invoke_endpoint(
        EndpointName="olive-test-vXX",  # your endpoint name
        ContentType="application/json",
        Body=json.dumps(payload),
    )

    raw_body = response["Body"].read().decode("utf-8")

    # 👇 Add this line to see EVERYTHING in Streamlit
    import streamlit as st
    st.subheader("Raw response from SageMaker (debug)")
    st.json(json.loads(raw_body))

    df = pd.read_json(raw_body)
    return df


In [75]:
import importlib
import inference

# If you change inference.py and rerun, this will reload it
importlib.reload(inference)

# Quick sanity checks
print("Has model_fn:", hasattr(inference, "model_fn"))
print("Has input_fn:", hasattr(inference, "input_fn"))
print("Has predict_fn:", hasattr(inference, "predict_fn"))
print("Has output_fn:", hasattr(inference, "output_fn"))

# Optional: test output_fn shape
print(inference.output_fn({"foo": "bar"}, "application/json"))


Has model_fn: True
Has input_fn: True
Has predict_fn: True
Has output_fn: True
('{"foo": "bar"}', 'application/json')


In [76]:
import boto3

MODEL_PACKAGE_ARN = (
    "arn:aws:sagemaker:us-east-1:006363897224:model-package/OliveForecastModelPackageGroup/1"
)

sm = boto3.client("sagemaker")

mp = sm.describe_model_package(ModelPackageName=MODEL_PACKAGE_ARN)

model_artifact = mp["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
image_uri = mp["InferenceSpecification"]["Containers"][0]["Image"]

print("Model artifact:", model_artifact)
print("Image URI:", image_uri)


Model artifact: s3://olive-datalake-fall2025/mlops/training/pipelines-3j2htybfobfx-TrainModels-PXGIayyuK0/output/model.tar.gz
Image URI: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3


In [96]:
import sagemaker
from sagemaker.model import Model

session = sagemaker.Session()
role = sagemaker.get_execution_role()

ENDPOINT_NAME = "olive-test-v17"

model = Model(
    image_uri=image_uri,
    model_data=model_artifact,
    role=role,
    entry_point="inference.py",  # now the *copied, fixed* file
    sagemaker_session=session,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=ENDPOINT_NAME,
)


------!

In [97]:
response = runtime.invoke_endpoint(
    EndpointName="olive-test-v17",
    ContentType="application/json",
    Body=json.dumps(payload),
)

print(response["Body"].read().decode("utf-8"))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = runtime.invoke_endpoint(                                                          │
│   2 │   EndpointName="olive-test-v17",                                                           │
│   3 │   ContentType="application/json",                                                          │
│   4 │   Body=json.dumps(payload),                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) 
from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is 
overloaded or there is an error in the application.</p>
". See 
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpo
ints/olive-test-v17 in account 006363897224 for more information.

In [1]:
import io
import json
import boto3
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# Assume these already exist in your notebook:
# - model_dict  (from model_fn)
# - FEATURE_COLS
# - _create_features  (same as in inference.py)


def predict_fn_with_country(input_data, model_dict):
    """
    Extended version of predict_fn that supports optional 'country' and 'grade'.

    input_data keys:
      - data_s3_path: str (e.g. "s3://.../processed.parquet")
      - model_name: str (e.g. "global_ridge")
      - steps: int
      - country: optional str, e.g. "EL", "ES"
      - grade: optional str, e.g. "EVOO", "REFINED"
    """
    # 1. Download and read historical data from S3
    s3_path = input_data["data_s3_path"]
    if not s3_path.startswith("s3://"):
        raise ValueError(f"data_s3_path must be an s3:// URI, got {s3_path}")

    _, _, rest = s3_path.partition("s3://")
    bucket, _, key = rest.partition("/")

    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=bucket, Key=key)
    buf = io.BytesIO(obj["Body"].read())
    table = pq.read_table(buf)
    df = table.to_pandas()

    # --- NEW: optional filtering by country / grade -----------------
    country = input_data.get("country")
    grade = input_data.get("grade")

    if country is not None:
        df = df[df["country"] == country]

    if grade is not None:
        df = df[df["grade"] == grade]

    if df.empty:
        raise ValueError(
            f"No data found after filtering for country={country}, grade={grade}."
        )
    # ----------------------------------------------------------------

    # 2. Prepare time index and sort
    df["week_start"] = pd.to_datetime(df["week_start"])
    df = df.set_index("week_start").sort_index()

    model_name = input_data["model_name"]
    steps = int(input_data["steps"])

    if model_name not in model_dict:
        raise ValueError(f"Unknown model_name '{model_name}'")

    model = model_dict[model_name]

    preds = []
    temp = df.copy()
    current_index = temp.index.max()

    for _ in range(steps):
        next_week = current_index + pd.Timedelta(weeks=1)

        temp = _create_features(temp)
        temp = temp[~temp.index.duplicated(keep="last")]
        temp = temp.sort_index()

        X_next = temp.loc[[next_week], FEATURE_COLS]
        y_hat = float(model.predict(X_next)[0])

        preds.append(
            {
                "week_start": next_week.strftime("%Y-%m-%d"),
                "forecast": y_hat,
                "country": country,
                "grade": grade,
                "model_used": model_name,
            }
        )

        temp.loc[next_week, "price_usd_per_l"] = y_hat
        current_index = next_week

    return preds


In [2]:
test_input = {
    "data_s3_path": "s3://olive-datalake-fall2025/mlops/preprocess/processed.parquet",
    "model_name": "global_ridge",
    "steps": 8,
    "country": "EL",       # Greece
    "grade": "EVOO",       # Extra Virgin
}

preds = predict_fn_with_country(test_input, model_dict)
pd.DataFrame(preds).head()


NameError: name 'model_dict' is not defined

## This sections tests is the endpoint is working but requires a list of dictionaries schema
#### this confirm works for the version saved in require_file_inf_v1.py under user-default-efs/olive-forecasting/scripts

In [8]:
# ---------------------------
# Step 1: Check that your data is displaying correctly 
# ---------------------------
import pandas as pd

# Path to your Parquet in S3
s3_path = "s3://olive-datalake-fall2025/mlops/preprocess/processed.parquet"

# Read the first few rows
df = pd.read_parquet(s3_path, engine="pyarrow").head(5)

# Convert to JSON for endpoint
payload = df.to_json(orient="records")

print(payload)

[{"week_start":1728777600000,"price_usd_per_l":5.3807423581,"cost_pressure":0.3050832196,"lag1week":null,"lag2week":null,"rolling3":5.3807423581,"rolling10":5.3807423581,"month":10,"dayofweek":6,"quarter":4,"sin_week":-0.9709418174,"country":"EL","market":"GREECE","grade":"REFINED"},{"week_start":1281225600000,"price_usd_per_l":3.0915163755,"cost_pressure":0.0140133591,"lag1week":null,"lag2week":null,"rolling3":3.0915163755,"rolling10":3.0915163755,"month":8,"dayofweek":6,"quarter":3,"sin_week":-0.5680647467,"country":"EL","market":"National","grade":"EVOO"},{"week_start":1281830400000,"price_usd_per_l":3.0073442149,"cost_pressure":0.0140133591,"lag1week":3.0915163755,"lag2week":null,"rolling3":3.0494302952,"rolling10":3.0494302952,"month":8,"dayofweek":6,"quarter":3,"sin_week":-0.6631226582,"country":"EL","market":"National","grade":"EVOO"},{"week_start":1282435200000,"price_usd_per_l":3.052190393,"cost_pressure":0.0140133591,"lag1week":3.0073442149,"lag2week":3.0915163755,"rolling3":

In [7]:
# ---------------------------
# Step 2: Create SageMaker runtime client
# ---------------------------
runtime = boto3.client('sagemaker-runtime')

# ---------------------------
# Step 3: Call your endpoint
# ---------------------------
endpoint_name = "olive-test-v2"  # replace with the deployed endpoint
response = runtime.invoke_endpoint(
    EndpointName= endpoint_name,
    ContentType="application/json",
    Body=payload
)

# ---------------------------
# Step 4: Read the response
# ---------------------------
result = response['Body'].read().decode('utf-8')
predictions = json.loads(result)

print(predictions)

{'predictions': [10.789956716553682, 5.999068199587563, 3.3543985577992763, 3.05806429020332, 2.9845484771701276]}


### Test that tries to keep the data within inference instead of called explicitly 
#### NOT CURRENTLY WORKING

In [9]:
import pandas as pd
import boto3
import json

# --- CONFIG ---
ENDPOINT_NAME = "olive-test-v2"
S3_PARQUET = "s3://olive-datalake-fall2025/mlops/preprocess/processed.parquet"

# --- Load a small slice of your processed data ---
df = pd.read_parquet(S3_PARQUET, engine="pyarrow").head(10)

# Convert to list-of-dicts JSON
records_json = df.to_json(orient="records")

# --- Choose your model + step ---
request_payload = {
    "model_name": "country_grade_PT_LAMPANTE",   # ← TRY A SPECIFIC MODEL
    "steps": 12,                                 # ← TRY A SPECIFIC FORECAST HORIZON
    "data": json.loads(records_json)             # ← Embed your data records
}

# --- Call the endpoint ---
sm_runtime = boto3.client("sagemaker-runtime")

response = sm_runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(request_payload)
)

# --- Read prediction result ---
result = json.loads(response["Body"].read())
print("Predictions:", result)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 # --- Call the endpoint ---                                                                 │
│   23 sm_runtime = boto3.client("sagemaker-runtime")                                              │
│   24                                                                                             │
│ ❱ 25 response = sm_runtime.invoke_endpoint(                                                      │
│   26 │   EndpointName=ENDPOINT_NAME,                                                             │
│   27 │   ContentType="application/json",                                                         │
│   28 │   Body=json.dumps(request_payload)                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) 
from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is 
overloaded or there is an error in the application.</p>
". See 
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpo
ints/olive-test-v2 in account 006363897224 for more information.

## This section is to check logs when you don't understand the error

In [7]:
import boto3

# --- CONFIG ---
endpoint_name = "olive-test-v2"  # your endpoint
region = "us-east-1"

logs_client = boto3.client("logs", region_name=region)

# CloudWatch log group for SageMaker endpoints
log_group = f"/aws/sagemaker/Endpoints/{endpoint_name}"

# 1. Get the most recent log streams
streams = logs_client.describe_log_streams(
    logGroupName=log_group,
    orderBy="LastEventTime",
    descending=True,
    limit=1
)

if not streams['logStreams']:
    print("No log streams found for this endpoint.")
else:
    log_stream_name = streams['logStreams'][0]['logStreamName']
    
    # 2. Get log events
    events = logs_client.get_log_events(
        logGroupName=log_group,
        logStreamName=log_stream_name,
        startFromHead=True
    )

    # 3. Print the logs
    for event in events['events']:
        print(event['message'])


/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-11-25 05:02:22,770 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2025-11-25 05:02:22,774 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2025-11-25 05:02:22,775 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
 

In [2]:
#Just Brandon looking at the exact dataframe for the UI pull

import pandas as pd

df = pd.read_parquet("s3://olive-datalake-fall2025/mlops/preprocess/processed.parquet")
df.columns.tolist()


['week_start',
 'price_usd_per_l',
 'cost_pressure',
 'lag1week',
 'lag2week',
 'rolling3',
 'rolling10',
 'month',
 'dayofweek',
 'quarter',
 'sin_week',
 'country',
 'market',
 'grade']

In [3]:
df.shape
df.columns.tolist()
df.dtypes


week_start         datetime64[ns]
price_usd_per_l           float64
cost_pressure             float64
lag1week                  float64
lag2week                  float64
rolling3                  float64
rolling10                 float64
month                       int64
dayofweek                   int64
quarter                     int64
sin_week                  float64
country                    object
market                     object
grade                      object
dtype: object

In [4]:
df.head(1).to_dict(orient="records")


[{'week_start': Timestamp('2024-10-13 00:00:00'),
  'price_usd_per_l': 5.380742358078603,
  'cost_pressure': 0.30508321961052887,
  'lag1week': nan,
  'lag2week': nan,
  'rolling3': 5.380742358078603,
  'rolling10': 5.380742358078603,
  'month': 10,
  'dayofweek': 6,
  'quarter': 4,
  'sin_week': -0.9709418174260523,
  'country': 'EL',
  'market': 'GREECE',
  'grade': 'REFINED'}]

In [3]:
import pandas as pd
import boto3
import io  # NEW

path = "s3://olive-datalake-fall2025/mlops/preprocess/processed.parquet"

# Parse bucket + key
_, _, rest = path.partition("s3://")
bucket, _, key = rest.partition("/")

s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket, Key=key)

# Read the streaming body into a BytesIO buffer
buffer = io.BytesIO(obj["Body"].read())

df = pd.read_parquet(buffer)  # now it's seekable
print(df.columns)
print(df.head())


Index(['week_start', 'price_usd_per_l', 'cost_pressure', 'lag1week',
       'lag2week', 'rolling3', 'rolling10', 'month', 'dayofweek', 'quarter',
       'sin_week', 'country', 'market', 'grade'],
      dtype='object')
  week_start  price_usd_per_l  cost_pressure  lag1week  lag2week  rolling3  \
0 2024-10-13         5.380742       0.305083       NaN       NaN  5.380742   
1 2010-08-08         3.091516       0.014013       NaN       NaN  3.091516   
2 2010-08-15         3.007344       0.014013  3.091516       NaN  3.049430   
3 2010-08-22         3.052190       0.014013  3.007344  3.091516  3.050350   
4 2010-08-29         2.942624       0.014013  3.052190  3.007344  3.000720   

   rolling10  month  dayofweek  quarter  sin_week country    market    grade  
0   5.380742     10          6        4 -0.970942      EL    GREECE  REFINED  
1   3.091516      8          6        3 -0.568065      EL  National     EVOO  
2   3.049430      8          6        3 -0.663123      EL  National     EVOO